## Document classification con embedding glove

In [26]:
#Import and constants
import math
import os
import glob
import random
import nltk
import numpy as np
import math
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

BETA=16
GAMMA=4
SIMILARITY_TRESHOLD = 0.5

lemmatizer = nltk.WordNetLemmatizer()

In [27]:
#Utilities

#Carica i documenti fornendo il nome della cartella che contiene le cartelle dei singoli topic
#Ogni documento viene taggato con il nome della cartella in cui risiede, questo rappresenta la sua classe
def load_datasets(parent_folder_name):
    dataset = []
    # Get a list of all items in the folder
    items = os.listdir(parent_folder_name)
    # Filter out folders from the list
    folder_list = [item for item in items if os.path.isdir(os.path.join(parent_folder_name, item))]
    for folder_name in folder_list:
        file_list = glob.glob(os.path.join(os.path.join(parent_folder_name, folder_name), "*"))

        # Iterate over each file
        for file_path in file_list:
            with open(file_path, "r") as file:
                content = file.read()
                dataset.append([content, folder_name])
    return dataset

#Fornisce l'embedding di un termine dato glove, in caso il termine non sia trovato fornisce 0
def get_embedding(key, glove):
    if key in glove:
        return glove.get_vector(key)
    else:
        return np.zeros(300)  # Return a zero vector if the key is not found

#Esegue l'embedding di ogni documento del dataset
def embed_dataset(dataset, glove):
    #Per ogni tupla (Documento,Classificazione)
    for tuple in dataset:
        #Embeddo il documento
        tuple[0] = embed_document(tuple[0], glove)
    return dataset

#Calcola embedding del documento come la media degli embedding di tutte le parole tokenizzate
def embed_document(document, glove):
    # Tokenize the sentence into words
    words = nltk.word_tokenize(document.lower())

    # Embed each word and calculate the average embedding
    embeddings = [get_embedding(word, glove) for word in words]
    return np.mean(embeddings, axis=0)
#Funzione di filtraggio che fornisce tutti i documenti di un dataset aventi una specifica label
def get_dataset_elements_with_label(dataset, label):
    elements = []
    for elem in dataset:
        if (elem[1] == label):
            elements.append(elem)
    return elements

# Caricamento del dataset e di glove

In [28]:
#Caricamento del dataset
dataset = load_datasets("data-es4/data/20_NGs_400")
#Caricamento di glove
glove = KeyedVectors.load_word2vec_format("glove.6B/glove.6B.300d.txt", no_header=True)

# Embedding del dataset

In [29]:
dataset_embedded = embed_dataset(dataset, glove)

C:\Users\Cavag\PycharmProjects\Natural_Language_Techniques\venv\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Cavag\PycharmProjects\Natural_Language_Techniques\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Divisione del dataset in training set e test set

Decidiamo di lavorare classe per classe per creare un training set bilanciato

In [30]:
random.seed(22)
#Calcolo l'elenco di tutte le label presenti nel dataset
labels_set = set([t[1] for t in dataset_embedded])
#Creo le strutture dati
x_train_data = []
y_train_data = []
x_test_data = []
y_test_data = []
#Per ogni possibile classe
for label in labels_set:
    #Estraggo dal dataset gli elementi con quella classe
    label_dataset = get_dataset_elements_with_label(dataset_embedded, label)
    #Li divido in training set e test set
    train_set, test_set = train_test_split(label_dataset, test_size=0.1, random_state=19)
    #Li aggiungo alle strutture dati
    for elem in train_set:
        if(isinstance(elem[0], np.ndarray)):
            x_train_data.append(elem[0])
            y_train_data.append(elem[1])
    for elem in test_set:
         if(isinstance(elem[0], np.ndarray)):
            x_test_data.append(elem[0])
            y_test_data.append(elem[1])

# Implementazione metodo Rocchio senza Near Positives

In [31]:
#Metodo usato per estrarre dal dataset delle feature tutti gli elementi con una label (Usato per calcolare i Positives examples)
#Features e Labels sono array collegati, features[0] sono le features dell'esempio 0 che avrà label label[0]
def get_features_with_same_label(features, labels, label):
    correct_features = []
    for i, elem in enumerate(labels):
        if (elem == label):
            correct_features.append(features[i])
    return correct_features

#Metodo usato per estrarre dal dataset delle feature tutti gli elementi non aventi una label (Usato per calcolare i Negatives examples)
#Features e Labels sono array collegati, features[0] sono le features dell'esempio 0 che avrà label label[0]
def get_features_with_different_label(features, labels, label):
    correct_features = []
    for i, elem in enumerate(labels):
        if (elem != label):
            correct_features.append(features[i])
    return correct_features

#Metodo per il training del modello
def train_rocchio(features, labels, beta, gamma):
    #Calcolo le labels che andrò a modellare
    possible_labels = set(labels)
    model = []
    #Per ogni classe
    for label in possible_labels:
        #Calcolo gli esempi positives
        positive_examples = get_features_with_same_label(features, labels, label)
        #Calcolo gli esempi negatives
        negative_examples = get_features_with_different_label(features, labels, label)
        #Calcolo le medie
        positive_examples = beta * np.mean(positive_examples, axis=0)
        negative_examples = gamma * np.mean(negative_examples, axis=0)
        #Calcolo il modello per la classe
        model.append(((positive_examples - negative_examples), label))
    return model

#Metodo per predire un elemento dato il modello
def predict(elem, model):
    best_predict = None
    best_similarity = -1
    #Per ogni possibile classe
    for label in model:
        #Calcolo la similarità tra il modello della classe e l'mebedding dell'elemento
        similarity = cosine_similarity(elem.reshape(1, -1), label[0].reshape(1, -1))[0]
        #Salvo la similarità migliore e la ritorno
        if (best_similarity == -1):
            best_similarity = similarity
            best_predict = label[1]
        elif (similarity > best_similarity):
            best_similarity = similarity
            best_predict = label[1]
    return best_predict, best_similarity

# Training e test del modello rocchio

In [32]:
#Alleno il modello
model = train_rocchio(x_train_data, y_train_data, BETA, GAMMA)
hit = 0
#Per ogni modello del test set verifico le predizioni
for i, elem in enumerate(x_test_data):
    predicted_value, sim = predict(elem, model)
    if (predicted_value == y_test_data[i]):
        hit += 1
    else:
        print("ERROR, EXPECTED", y_test_data[i], "GOT", predicted_value, "WITH SIM: ", sim)
#Calcolo accuratezza
if(hit==0):
    prec=0
else:
    perc=(hit / len(x_test_data))*100
print("Accuracy:", perc, "%", "N_tests:",len(x_test_data),"Hits:",hit)

ERROR, EXPECTED sci.crypt GOT comp.sys.mac.hardware WITH SIM:  [0.96002894]
ERROR, EXPECTED sci.crypt GOT comp.windows.x WITH SIM:  [0.97191266]
ERROR, EXPECTED sci.med GOT talk.religion.misc WITH SIM:  [0.9497588]
ERROR, EXPECTED misc.forsale GOT comp.windows.x WITH SIM:  [0.91607043]
ERROR, EXPECTED misc.forsale GOT talk.politics.guns WITH SIM:  [0.97130805]
ERROR, EXPECTED comp.sys.ibm.pc.hardware GOT comp.windows.x WITH SIM:  [0.91819115]
ERROR, EXPECTED comp.sys.ibm.pc.hardware GOT comp.graphics WITH SIM:  [0.97858118]
ERROR, EXPECTED comp.windows.x GOT sci.crypt WITH SIM:  [0.98133192]
ERROR, EXPECTED comp.windows.x GOT comp.os.ms-windows.misc WITH SIM:  [0.98372821]
ERROR, EXPECTED talk.religion.misc GOT sci.crypt WITH SIM:  [0.97989356]
ERROR, EXPECTED comp.os.ms-windows.misc GOT comp.windows.x WITH SIM:  [0.93806999]
ERROR, EXPECTED comp.os.ms-windows.misc GOT sci.crypt WITH SIM:  [0.97973088]
ERROR, EXPECTED rec.sport.baseball GOT sci.med WITH SIM:  [0.9716577]
ERROR, EXPECTE

# Modello rocchio con near positives
Si è proposto di calcolare i near positives come gli elementi che hanno label diversa e una distanza media dagli esempi posivi minore di SIMILARITY_TRESHOLD

In [33]:
#Calcola la similarità media tra un esempio negativo e una lista di esempi positivi
def calc_avg_similarity(positives, feature):
    sims = []
    for elem in positives:
        sims.append(cosine_similarity(elem.reshape(1, -1), feature.reshape(1, -1))[0])
    return np.mean(sims)

#Calcola l'elenco dei near positives dato una label corretta
def get_features_with_nears_positives(features, positives, labels, label):
    correct_features = []
    for i, elem in enumerate(labels):
        if (elem != label):
            sim = calc_avg_similarity(positives,features[i])
            if (sim >= SIMILARITY_TRESHOLD):
                correct_features.append(features[i])
    return correct_features

#Allena il modello sfruttando i near positives
def train_rocchio_with_near_positives(features, labels, beta, gamma):
    possible_labels = set(labels)
    model = []
    #Per ogni classe
    for label in possible_labels:
        #Calcolo gli esempi positives
        positive_examples = get_features_with_same_label(features, labels, label)
        #Calcolo i near positives
        near_positives_examples = get_features_with_nears_positives(features, positive_examples, labels, label)
        #Faccio le medie
        positive_examples = beta * np.mean(positive_examples, axis=0)
        near_positives_examples = gamma * np.mean(near_positives_examples, axis=0)
        #Calcolo il modello
        model.append(((positive_examples - near_positives_examples), label))
    return model

# Training e test del modello rocchio con near positives

In [34]:
#Alleno il modello
model = train_rocchio_with_near_positives(x_train_data, y_train_data, 16, 4)
hit = 0
#Per ogni modello del test set verifico le predizioni
for i, elem in enumerate(x_test_data):
    predicted_value, sim = predict(elem, model)
    if (predicted_value == y_test_data[i]):
        hit += 1
    else:
        print("ERROR, EXPECTED", y_test_data[i], "GOT", predicted_value, "WITH SIM: ", sim)
#Calcolo accuratezza
if(hit==0):
    prec=0
else:
    perc=(hit / len(x_test_data))*100
print("Accuracy:", perc, "%", "N_tests:",len(x_test_data),"Hits:",hit)

ERROR, EXPECTED sci.crypt GOT comp.sys.mac.hardware WITH SIM:  [0.96010116]
ERROR, EXPECTED sci.crypt GOT comp.windows.x WITH SIM:  [0.97192239]
ERROR, EXPECTED sci.med GOT talk.religion.misc WITH SIM:  [0.94983564]
ERROR, EXPECTED misc.forsale GOT comp.windows.x WITH SIM:  [0.91604151]
ERROR, EXPECTED misc.forsale GOT talk.politics.guns WITH SIM:  [0.97126263]
ERROR, EXPECTED comp.sys.ibm.pc.hardware GOT comp.windows.x WITH SIM:  [0.91835666]
ERROR, EXPECTED comp.sys.ibm.pc.hardware GOT comp.graphics WITH SIM:  [0.97850905]
ERROR, EXPECTED comp.windows.x GOT sci.crypt WITH SIM:  [0.98127539]
ERROR, EXPECTED comp.windows.x GOT comp.os.ms-windows.misc WITH SIM:  [0.9837652]
ERROR, EXPECTED talk.religion.misc GOT sci.crypt WITH SIM:  [0.97990536]
ERROR, EXPECTED comp.os.ms-windows.misc GOT comp.windows.x WITH SIM:  [0.93814874]
ERROR, EXPECTED comp.os.ms-windows.misc GOT sci.crypt WITH SIM:  [0.97969467]
ERROR, EXPECTED rec.sport.baseball GOT sci.med WITH SIM:  [0.97165375]
ERROR, EXPECT